## Using BART for Like Prediction with Text Only Input

## Data Preprocessing

### Read File

In [1]:
import pandas as pd
import numpy as np
train = pd.read_csv('../../raw_data/intern_homework_train_dataset.csv')
test = pd.read_csv('../../raw_data/intern_homework_public_test_dataset.csv')
train.head()

,title,created_at,like_count_1h,like_count_2h,like_count_3h,like_count_4h,like_count_5h,like_count_6h,comment_count_1h,comment_count_2h,comment_count_3h,comment_count_4h,comment_count_5h,comment_count_6h,forum_id,author_id,forum_stats,like_count_24h
0,我的排骨湯,2022-10-05 14:20:21 UTC,12,15,15,15,16,18,10,10,10,10,10,10,598518,428921,0.7,26
1,#請益 婚禮穿搭,2022-10-05 14:28:13 UTC,0,0,3,4,4,4,2,5,8,9,9,9,399302,650840,63.9,11
2,無謂的啦啦隊,2022-10-06 07:18:22 UTC,3,7,8,11,12,14,1,1,2,3,3,3,650776,717288,19.2,19
3,文學理論 課本,2022-09-20 11:39:14 UTC,2,7,11,24,26,26,2,2,8,32,38,63,471023,173889,7.9,29
4,一般課程,2022-09-05 10:18:24 UTC,3,7,7,10,10,11,15,26,35,38,48,49,230184,594332,36.2,16


In [2]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(train, random_state=777, train_size=0.9)
len(train), len(valid)

(45000, 5000)

### Feature Selection

挑選 Label

In [3]:
train_label = train['like_count_24h']
valid_label = valid['like_count_24h']
test_label = test['like_count_24h']
train_label

35094     15
13209      8
10950     15
40900      6
27226    266
        ... 
26695    224
36785     10
40535     11
15931     10
47919     34
Name: like_count_24h, Length: 45000, dtype: int64

In [4]:
train_label = train_label.tolist()
valid_label = valid_label.tolist()
test_label = test_label.tolist()
train_label[0]

15

In [5]:
train_label = list(map(str, train_label))
valid_label = list(map(str, valid_label))
test_label = list(map(str, test_label))
train_label[0]

'15'

因為 BART 為自然語言模型，label 也需要為文字的格式

挑選 Input

省略掉 作者 ID / 看板 ID  因為我認為 BART 沒辦法了解這些 Feature, 會讓模型預測文字輸出帶來 noise.

In [6]:
# 指定要刪除的 column names，並使用 drop 函數將這些 column 刪除
drop_columns = ['author_id', 'like_count_24h', 'forum_id']

train_input = train.drop(drop_columns, axis=1)
valid_input = valid.drop(drop_columns, axis=1)
test_input = test.drop(drop_columns, axis=1)

### Data Transfromations

處理 created_by Feature

In [7]:
# 將文章發佈時間 拆成 星期幾 與 小時 的函數
def split_date(df, date_column):

    # 將 created_by 欄位轉換成日期格式
    df[date_column] = pd.to_datetime(df[date_column], utc=True)
    
    # 新增 星期幾 和 小時 欄位
    df['weekday'] = df[date_column].dt.weekday
    df['hour'] = df[date_column].dt.hour

    # 移除 created_by 欄位
    df = df.drop(date_column, axis=1)

    # 回傳處理過的資料集
    return df

In [8]:
train_input = split_date(train_input, 'created_at')
valid_input = split_date(valid_input, 'created_at')
test_input = split_date(test_input, 'created_at')

# 顯示處理過的資料集
train_input.head()

,title,like_count_1h,like_count_2h,like_count_3h,like_count_4h,like_count_5h,like_count_6h,comment_count_1h,comment_count_2h,comment_count_3h,comment_count_4h,comment_count_5h,comment_count_6h,forum_stats,weekday,hour
35094,111台聯A3台綜A10轉學考上榜心得,1,5,7,8,9,10,2,2,6,7,7,7,41.5,5,12
13209,［世界志工社｜第二次招生說明會預告］,7,7,7,7,7,7,21,22,22,22,22,22,59.9,6,13
10950,「攜手」-我們在身份轉換中的震盪,3,3,3,3,3,4,5,5,5,5,5,5,0.1,5,12
40900,怎麼抓住小一女生的心,0,0,0,0,0,0,1,4,4,4,4,5,2.4,1,12
27226,楊丞琳以前家境清寒吃不起海鮮,2,3,5,5,5,5,0,0,1,1,1,1,359.8,1,16


In [9]:
train_input.columns

Index(['title', 'like_count_1h', 'like_count_2h', 'like_count_3h',
       'like_count_4h', 'like_count_5h', 'like_count_6h', 'comment_count_1h',
       'comment_count_2h', 'comment_count_3h', 'comment_count_4h',
       'comment_count_5h', 'comment_count_6h', 'forum_stats', 'weekday',
       'hour'],
      dtype='object')

自訂義 dict，將 weekday 轉換為中文星期幾

In [10]:
weekday_dict = {
    0: '星期一',
    1: '星期二',
    2: '星期三',
    3: '星期四',
    4: '星期五',
    5: '星期六',
    6: '星期日'
}

# 將 weekday 轉換為中文星期幾
train_input['weekday'] = train_input['weekday'].map(weekday_dict)
valid_input['weekday'] = valid_input['weekday'].map(weekday_dict)
test_input['weekday'] = test_input['weekday'].map(weekday_dict)

將 Input 的數值與文字合併一篇文章來當作 BART Input

In [11]:
# 將 dataframe 內的 feature 合併成一串文字 的函數
def transfrom_to_text(df):

    passage = ""

    # combined title and post time
    passage += "這篇文章標題是 {} ，在{}{}點發佈。".format(df["title"], df["weekday"], df["hour"])

    # combined forum_stats
    passage += "這篇發佈的看板資訊為 {} 。".format(df["forum_stats"])

    # combined likes_count
    passage += "文章在發佈後的第一小時累積到的愛心數有 {}，在第二小時累積到的愛心數有 {}，".format(df["like_count_1h"], df["like_count_2h"])
    passage += "第三小時累積到的愛心數有 {}，在第四小時累積到的愛心數有 {}，".format(df["like_count_3h"], df["like_count_4h"])
    passage += "第五小時累積到的愛心數有 {}，在第六小時累積到的愛心數有 {}。".format(df["like_count_5h"], df["like_count_6h"])

    # combined comment_count
    passage += "文章在發佈後的第一小時累積到的留言數有 {}，在第二小時累積到的留言數有 {}，".format(df["comment_count_1h"], df["comment_count_2h"])
    passage += "第三小時累積到的留言數有 {}，在第四小時累積到的留言數有 {}，".format(df["comment_count_3h"], df["comment_count_4h"])
    passage += "第五小時累積到的留言數有 {}，在第六小時累積到的留言數有 {}。".format(df["comment_count_5h"], df["comment_count_6h"])

    
    return passage

In [12]:
train_text = train_input.apply(transfrom_to_text, axis=1).tolist()
valid_text = valid_input.apply(transfrom_to_text, axis=1).tolist()
test_text = test_input.apply(transfrom_to_text, axis=1).tolist()
train_text[0]

'這篇文章標題是 111台聯A3台綜A10轉學考上榜心得 ，在星期六12點發佈。這篇發佈的看板資訊為 41.5 。文章在發佈後的第一小時累積到的愛心數有 1，在第二小時累積到的愛心數有 5，第三小時累積到的愛心數有 7，在第四小時累積到的愛心數有 8，第五小時累積到的愛心數有 9，在第六小時累積到的愛心數有 10。文章在發佈後的第一小時累積到的留言數有 2，在第二小時累積到的留言數有 2，第三小時累積到的留言數有 6，在第四小時累積到的留言數有 7，第五小時累積到的留言數有 7，在第六小時累積到的留言數有 7。'

In [13]:
print("Train Data 總共有 {} 筆".format(len(train_text)))
print("Valid Data 總共有 {} 筆".format(len(valid_text)))
print("Test Data 總共有 {} 筆".format(len(test_text)))

Train Data 總共有 45000 筆
Valid Data 總共有 5000 筆
Test Data 總共有 10000 筆


### Tokenization

In [14]:
from transformers import BertTokenizer
import torch

2023-04-08 11:32:09.844828: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-08 11:32:09.947271: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-08 11:32:10.424847: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvrtc.so.11.0: cannot open shared object file: No such file or directory
2023-04-08 11:32:10.424980: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvrtc.so.11.0: cannot open shared object file: No such file or direc

In [15]:
tokenizer = BertTokenizer.from_pretrained("fnlp/bart-base-chinese")

In [16]:
train_encodings = tokenizer(train_text ,text_target = train_label, truncation=True, padding=True)
valid_encodings = tokenizer(valid_text ,text_target = valid_label, truncation=True, padding=True)
test_encodings = tokenizer(test_text ,text_target = test_label, truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [17]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

## Fine-tuning

In [18]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch

model = AutoModelForSeq2SeqLM.from_pretrained("fnlp/bart-base-chinese")

In [19]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = Dataset(train_encodings)
valid_dataset = Dataset(valid_encodings)
test_dataset = Dataset(test_encodings)

In [20]:
batch_size = 16
args = Seq2SeqTrainingArguments(
    output_dir = "./results",
    save_strategy = "epoch",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="MAPE",
    greater_is_better = False,
    weight_decay=0.01,
    predict_with_generate=True,
    eval_accumulation_steps = 1,
)

In [21]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [22]:
import numpy as np
import re
from sklearn.metrics import mean_absolute_percentage_error
def compute_metrics(p):
    predictions, labels = p
    
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # store all sentences
    predicted = []
    true_label = []
    for k in range(len(decoded_labels)):

        match = re.search(r'\d+', decoded_preds[k])
        if match:
            pred = int(match.group())
        label = int(decoded_labels[k])

        predicted.append(pred)
        true_label.append(label)

    # evaluation metrics
    MAPE = mean_absolute_percentage_error(true_label, predicted)
    
    result = {'MAPE': MAPE}
    
    return result

In [23]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [24]:
trainer.train()

/user_data/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 45000
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 4221
  Number of trainable parameters = 140193024
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hankystyle. Use `wandb login --relogin` to force relogin


The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: token_type_ids. If token_type_ids are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
/user_data/.local/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Mape
1,0.866500,0.840436,0.317031
2,0.825100,0.816238,0.300382
3,0.807900,0.811054,0.297469


***** Running Evaluation *****
  Num examples = 5000
  Batch size = 32
The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: token_type_ids. If token_type_ids are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
Generate config GenerationConfig {
  "bos_token_id": 101,
  "decoder_start_token_id": 102,
  "early_stopping": true,
  "eos_token_id": 102,
  "forced_eos_token_id": 102,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

  Num examples = 5000
  Batch size = 32
The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: token_type_ids. If token_type_ids are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
Generate config GenerationConfig {
  "bos_token_id": 101,
  "

TrainOutput(global_step=4221, training_loss=0.8582116230718617, metrics={'train_runtime': 1681.3066, 'train_samples_per_second': 80.295, 'train_steps_per_second': 2.511, 'total_flos': 3.59321743872e+16, 'train_loss': 0.8582116230718617, 'epoch': 3.0})

## Evaluation

Save Model

In [ ]:
trainer.save_model('saved_model/bart-base-text2text-dcard-forum_stats')

Saving model checkpoint to saved_model/bart-base-text2text-dcard-forum_stats
Configuration saved in saved_model/bart-base-text2text-dcard-forum_stats/config.json
Configuration saved in saved_model/bart-base-text2text-dcard-forum_stats/generation_config.json
Model weights saved in saved_model/bart-base-text2text-dcard-forum_stats/pytorch_model.bin
tokenizer config file saved in saved_model/bart-base-text2text-dcard-forum_stats/tokenizer_config.json
Special tokens file saved in saved_model/bart-base-text2text-dcard-forum_stats/special_tokens_map.json


## Prediction

In [27]:
import pandas as pd
import numpy as np
private_test = pd.read_csv('../../raw_data/intern_homework_private_test_dataset.csv')

In [28]:
# 指定要刪除的 column names，並使用 drop 函數將這些 column 刪除
drop_columns = ['author_id', 'forum_id']
private_test_input = private_test.drop(drop_columns, axis=1)

In [29]:
# 將文章發佈時間 拆成 星期幾 與 小時 的函數
def split_date(df, date_column):

    # 將 created_by 欄位轉換成日期格式
    df[date_column] = pd.to_datetime(df[date_column], utc=True)
    
    # 新增 星期幾 和 小時 欄位
    df['weekday'] = df[date_column].dt.weekday
    df['hour'] = df[date_column].dt.hour

    # 移除 created_by 欄位
    df = df.drop(date_column, axis=1)

    # 回傳處理過的資料集
    return df

In [30]:
private_test_input = split_date(private_test_input, 'created_at')

weekday_dict = {
    0: '星期一',
    1: '星期二',
    2: '星期三',
    3: '星期四',
    4: '星期五',
    5: '星期六',
    6: '星期日'
}

# 將 weekday 轉換為中文星期幾
private_test_input['weekday'] = private_test_input['weekday'].map(weekday_dict)

In [31]:
# 將 dataframe 內的 feature 合併成一串文字 的函數
def transfrom_to_text(df):

    passage = ""

    # combined title and post time
    passage += "這篇文章標題是 {} ，在{}{}點發佈。".format(df["title"], df["weekday"], df["hour"])

    # combined forum_stats
    passage += "這篇發佈的看板資訊為 {} 。".format(df["forum_stats"])

    # combined likes_count
    passage += "文章在發佈後的第一小時累積到的愛心數有 {}，在第二小時累積到的愛心數有 {}，".format(df["like_count_1h"], df["like_count_2h"])
    passage += "第三小時累積到的愛心數有 {}，在第四小時累積到的愛心數有 {}，".format(df["like_count_3h"], df["like_count_4h"])
    passage += "第五小時累積到的愛心數有 {}，在第六小時累積到的愛心數有 {}。".format(df["like_count_5h"], df["like_count_6h"])

    # combined comment_count
    passage += "文章在發佈後的第一小時累積到的留言數有 {}，在第二小時累積到的留言數有 {}，".format(df["comment_count_1h"], df["comment_count_2h"])
    passage += "第三小時累積到的留言數有 {}，在第四小時累積到的留言數有 {}，".format(df["comment_count_3h"], df["comment_count_4h"])
    passage += "第五小時累積到的留言數有 {}，在第六小時累積到的留言數有 {}。".format(df["comment_count_5h"], df["comment_count_6h"])

    
    return passage

In [32]:
private_test_text = private_test_input.apply(transfrom_to_text, axis=1).tolist()
private_test_text[0]

'這篇文章標題是 #心情 頂樓風好大 ，在星期四0點發佈。這篇發佈的看板資訊為 11.0 。文章在發佈後的第一小時累積到的愛心數有 6，在第二小時累積到的愛心數有 9，第三小時累積到的愛心數有 10，在第四小時累積到的愛心數有 12，第五小時累積到的愛心數有 12，在第六小時累積到的愛心數有 16。文章在發佈後的第一小時累積到的留言數有 4，在第二小時累積到的留言數有 8，第三小時累積到的留言數有 9，在第四小時累積到的留言數有 11，第五小時累積到的留言數有 12，在第六小時累積到的留言數有 12。'

In [33]:
print("預測資料總共有 {} 筆".format(len(private_test_text)))

預測資料總共有 10000 筆


In [34]:
from transformers import BertTokenizer
import torch
tokenizer = BertTokenizer.from_pretrained("fnlp/bart-base-chinese")
private_test_encodings = tokenizer(private_test_text, truncation=True, padding=True)

loading file vocab.txt from cache at /user_data/.cache/huggingface/hub/models--fnlp--bart-base-chinese/snapshots/4e93f21dca95a07747f434b0f9fe5d49cacc0441/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /user_data/.cache/huggingface/hub/models--fnlp--bart-base-chinese/snapshots/4e93f21dca95a07747f434b0f9fe5d49cacc0441/special_tokens_map.json
loading file tokenizer_config.json from cache at /user_data/.cache/huggingface/hub/models--fnlp--bart-base-chinese/snapshots/4e93f21dca95a07747f434b0f9fe5d49cacc0441/tokenizer_config.json
loading configuration file config.json from cache at /user_data/.cache/huggingface/hub/models--fnlp--bart-base-chinese/snapshots/4e93f21dca95a07747f434b0f9fe5d49cacc0441/config.json
Model config BartConfig {
  "_name_or_path": "fnlp/bart-base-chinese",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "Bart

In [35]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

private_test_dataset = Dataset(private_test_encodings)

In [36]:
predictions, labels, metrics = trainer.predict(private_test_dataset)

***** Running Prediction *****
  Num examples = 10000
  Batch size = 32
The following columns in the test set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: token_type_ids. If token_type_ids are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
Generate config GenerationConfig {
  "bos_token_id": 101,
  "decoder_start_token_id": 102,
  "early_stopping": true,
  "eos_token_id": 102,
  "forced_eos_token_id": 102,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

  Num examples = 10000
  Batch size = 32
The following columns in the test set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: token_type_ids. If token_type_ids are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
Generate config GenerationConfig {
  "bos_token_id": 101,
  "decoder_st

In [37]:
metrics

{'test_runtime': 74.9939,
 'test_samples_per_second': 133.344,
 'test_steps_per_second': 4.174}

In [38]:
predictions

array([[  102,   101, 30920, ...,     0,     0,     0],
       [  102,   101,   125, ...,     0,     0,     0],
       [  102,   101,   125, ...,     0,     0,     0],
       ...,
       [  102,   101, 33146, ...,     0,     0,     0],
       [  102,   101, 31605, ...,     0,     0,     0],
       [  102,   101,   125, ...,     0,     0,     0]])

In [39]:
decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
decoded_preds

['21',
 '5',
 '5',
 '5',
 '15',
 '5',
 '5',
 '10',
 '5',
 '12',
 '26',
 '5',
 '5',
 '18',
 '94',
 '15',
 '7',
 '12',
 '9',
 '94',
 '10',
 '5',
 '12',
 '42',
 '5',
 '5',
 '26',
 '5',
 '9',
 '5',
 '5',
 '10',
 '5',
 '18',
 '14',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '23',
 '5',
 '15',
 '5',
 '5',
 '7',
 '26',
 '94',
 '41',
 '10',
 '6',
 '5',
 '9',
 '5',
 '5',
 '5',
 '6',
 '9',
 '7',
 '78',
 '5',
 '5',
 '12',
 '5',
 '5',
 '5',
 '11',
 '5',
 '18',
 '5',
 '7',
 '5',
 '5',
 '9',
 '5',
 '5',
 '5',
 '9',
 '10',
 '5',
 '14',
 '5',
 '12',
 '7',
 '138',
 '5',
 '10',
 '5',
 '9',
 '10',
 '45',
 '5',
 '5',
 '5',
 '5',
 '25',
 '5',
 '5',
 '5',
 '9',
 '5',
 '5',
 '5',
 '5',
 '15',
 '55',
 '9',
 '5',
 '5',
 '15',
 '5',
 '5',
 '21',
 '10',
 '81',
 '5',
 '14',
 '94',
 '5',
 '14',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '9',
 '41',
 '5',
 '12',
 '12',
 '5',
 '23',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '15',
 '5',
 '26',
 '57',
 '53',
 '15',
 '5',
 '5',
 '7',
 '12',
 '5',
 '5',
 '15',
 '5',
 '53',
 

Save Prediction

In [40]:
df = pd.DataFrame(decoded_preds, columns=["like_count_24h"])
df.to_csv("result(with_forum_stats).csv", index=False)